## Project Description

## Libraries

#### Python Integrated or in Folder

In [ ]:
import ipywidgets as widgets
import thinkdsp as thinkdsp
from thinkdsp import play_wave
from thinkdsp import read_wave
import math 
import soundfile as sf
import pyloudnorm as pyln
import pydub 
from pydub import AudioSegment
import spectrum
import scipy
import matplotlib.pyplot as plt

#### Require Installation

##### Enter these commands into your terminal:
    pip install soundfile
    pip install pyloudnorm
    pip install pydub
    pip install spectrum

## Variables

#### NOT Recommended for change

In [ ]:
ChooseBandCount = 10 # Standard Band Count
Controllrange = [-12, 12, 0] # [Decrease, Gain, Default] in dB
Normalise_Value = -15
EQBandmin = 5
EQBandmax = 20
EQstep = 5
ConfigBandCount = 21
OutputGain = -3

In [ ]:
Freqspecturmf0 = [16,32,64,128,256,512,1024,2048,4096,8192,0,0,0,0,0,0,0,0,0,0,0] # Displayed Frequency on EQ 
Freqspecturmfl = [0]*EQBandmax # Lower Frequency for EQ
Freqspecturmfh = [0]*EQBandmax # Higher Frequency for EQ

In [ ]:
# Generates Grid for EQ
# EQDisplay = widgets.GridspecLayout(ChooseBandCount, 1)

#### Recommended for change

In [ ]:
Text1 = widgets.Output()
with Text1:
    print("Choose your File, that you want to edit:")
    
Text2 = widgets.Output()
with Text2:
    print("\nChoose the focus of the EQ bandpass:")
    
Text3 = widgets.Output()
with Text3:
    print("\nChoose the EQ Slider Count:")
    
Text4 = widgets.Output()
with Text4:
    print("\nEqualizer:")
    
Text5 = widgets.Output()
with Text5:
    print("\nPress to start calculation:")

In [ ]:
UploadFileName = "Upload"
FileFormat = ".wav"
FinishedFileName = "Output"
CloneFileName = "Band"

## Functions

##### Generates EQ UI

In [ ]:
def Build_EQ (ChooseBandCount, Freqspecturmf0):
    
    slider = [0] * ChooseBandCount
    
    for i in range(ChooseBandCount):
        slider[i] = widgets.IntSlider(
           value=Controllrange[2],
            min=Controllrange[0],
            max=Controllrange[1],
            step=1,
            description=str(Freqspecturmf0[i])+' Hz',
            disabled=False,
            continuous_update=False,
            orientation='vertical',
            readout=True,
            readout_format='d',
            layout = widgets.Layout(width='50%')
        )
    
    return slider

##### Calculating Base Frequency of choosen Type of EQ 

In [ ]:
def returnbasefreq(SetFilterType, ChooseBandCount):
    # Scans input of choosen Type of EQ and gives back predefined start Frequencies for an 8 Band EQ
    if (SetFilterType == 'Bass'):
        f_base = 20
    if (SetFilterType == 'Mid'):
        f_base = 40
    if (SetFilterType == 'High'):
        f_base = 80
    
    # Modulates Frequency depending on difference between 8 / ChoosenBand
    f_base = f_base * (8/ChooseBandCount)
        
    return f_base

##### Calculates Base Frequencies that are displayed on EQ

In [ ]:
def calculate_frequencies(SetFilterType, ChooseBandCount):
    #resets values in Freqspecturm
    Freqspecturmf0 = [0] * (EQBandmax+1) # Clears all values that were precalculatet
    
    # Select start frequency for calculation
    f_base = int(returnbasefreq(SetFilterType, ChooseBandCount))
    # Calculates Modulation Rate for n-Frequencies
    f_mod = pow(2, 10 / ChooseBandCount)
    
    # Generates Freqspecturm for given Type and Bandcount
    Freqspecturmf0[0] = f_base
    
    for i in range(1, ChooseBandCount+1):
        Freqspecturmf0[i] = int(f_mod * Freqspecturmf0[i-1]) # int() Rounds Value to non decimal
    
    # print (Freqspecturm)
    return Freqspecturmf0

##### Calculates log average of f0[n] and f0[n-1]

In [ ]:
def calculate_frequencieslow(Freqspecturmf0, ChooseBandCount):
    # For the value of fl[0] of f0[0] we need to calculate f0[-1], fortunatly because we dont need a highpass on the first EQ we can set fl[0] = 0
    Freqspecturmfl = [0]*EQBandmax # Clears all values that were precalculatet
    
    Freqspecturmfl[0] = 0
    
    for i in range(1, ChooseBandCount):
            Freqspecturmfl[i] = int(math.sqrt(Freqspecturmf0[i] * Freqspecturmf0[i-1]))
    
    return Freqspecturmfl

In [ ]:
f = calculate_frequencieslow(Freqspecturmf0, ChooseBandCount)
f

##### Calculates log average of f0[n] and f0[n+1]

In [ ]:
def calculate_frequencieshigh(Freqspecturmf0, ChooseBandCount):
    # For the value of fh[n] of f0[n] we need to calculate f0[n+1]
    Freqspecturmfh = [0] * (EQBandmax + 1) # Clears all values that were precalculatet
    
    f_mod = pow(2, 10 / ChooseBandCount)
    fnp1 = f_mod * Freqspecturmf0[ChooseBandCount]
    
    for i in range(0, ChooseBandCount-1):
            Freqspecturmfh[i] = int(math.sqrt(Freqspecturmf0[i] * Freqspecturmf0[i+1]))
    
    # Since Upper Band needs COntroll too we need to define upper limit to max Frequency
    Freqspecturmfh[ChooseBandCount-1] = 48000
    
    return Freqspecturmfh

##### Normalising Audio

In [ ]:
# Value is the dB Value that the file will be normalised with
# void function: Overrides [FileName] File

def Normalise_Audio (FileName, FileFormat, Normalise):
    # Load Audio File as shape(data, wave)
    data, rate = sf.read(FileName + FileFormat)

    # Initiolising loudness Meter by BS.1770 standart 
    meter = pyln.Meter(rate) 
    # measure loudness in dB
    loudness = meter.integrated_loudness(data)
    
    # loudness normalize audio to [Normalise] dB LUFS
    loudness_normalized_audio = pyln.normalize.loudness(data, loudness, Normalise)
    sf.write(FileName + FileFormat, loudness_normalized_audio, rate)

##### Generating temporary files

In [ ]:
# Generates Temp Copies for Processing every single Band

def Generate_Copies(FileName, FileFormat, BandpassCount):
    # import Upload File
    wave = read_wave(UploadFileName + FileFormat)
    
    for n in range(0, BandpassCount):
        wave.write(FileName + str(n) + FileFormat)

##### Bandpass Filter 

In [ ]:
# Overwrites [n] temp Audio File
# Filename is the current edited Bandpass
# Gain is the current used Gain value of said bandpass

def Bandpass_Filter (FileName, FileFormat, n, Gain, fl, fh):
    # Load Audio File
    wave = read_wave(FileName + str(n) + FileFormat)

    spectrum = wave.make_spectrum()
    # Lowpass filter with fh
    spectrum.low_pass(fh)
    # Highpass filter with fl
    spectrum.high_pass(fl)
    # Overrides Temp Audio File
    filtered = spectrum.make_wave()
    filtered.write(FileName + str(n) + FileFormat)

    
    # Change Gain
    wave = AudioSegment.from_wav(FileName + str(n) + FileFormat)

    # but let's make him *very* quiet
    gain_wave = wave + Gain

    # save the output
    gain_wave.export(FileName + str(n) + FileFormat, "wav")

##### Build Finished Audio

In [ ]:
# Overlaps several AudioFiles which have the same base name + and adressing Number + ".wav"
# Example "FilterTemp" + string(n) + ".wav" starting from 0 to Bandpasscount-1
# FileName must not contail ".wav" !!!
# FileType is the file ending, for example ".wav"
# FinishedFileName is the Filename which the Audio will be saved. Must contain Filename+".wav"

def Build_Audio (FileName, FileFormat, FinishedFileName, BandpassCount):
    # Load first Adio file
    fullaudio = AudioSegment.from_wav(FileName + "0" + FileFormat)
    # Generates loop
    for i in range(1, ChooseBandCount):
        # import current file
        temp = AudioSegment.from_wav(FileName + str(i) + FileFormat)
        # Overlays fullaudio with new generated wave file
        fullaudio = fullaudio.overlay(temp)

    # Saves fullaudio
    fullaudio.export(FinishedFileName + ".wav", "wav")
    
    # Deletes temp Files (optional)

## Widgets

In [ ]:
Widget_Upload = widgets.FileUpload(
    accept= FileFormat,  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    tooltips=['Upload a .wav File to Edit it']
)

In [ ]:
Widget_Verify_Upload = widgets.Button(
    value=False,
    description='Submit',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [ ]:
Widget_Verify_EQ = widgets.Button(
    value=False,
    description='Calculate',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

In [ ]:
# Choose EQ Preset
Widget_SetFilterType = widgets.ToggleButtons(
    options=['Bass', 'Mid', 'High'],
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Focus on 200Hz', 'Focus on 1kHz', 'Focus on 4kHz']
)

In [ ]:
# Choose Band Count
Widget_ChooseBandCount = widgets.BoundedIntText(
    value=ChooseBandCount,
    min=EQBandmin,
    max=EQBandmax,
    step=EQstep,
    disabled=False
)

In [ ]:
progress = widgets.FloatProgress(
    value=0.0, 
    min=0.0, 
    max=1.0, 
    # description='Progress:',
    bar_style='success',
    # style={'bar_color': '#2EFF00'},
    orientation='horizontal'
    )

In [ ]:
Output = widgets.Output()

#### Combine all Widgets

In [ ]:
UserInterface = widgets.VBox(children = [Text1, Widget_Upload, Widget_Verify_Upload, Text2, Widget_SetFilterType, Text3, Widget_ChooseBandCount, Text4])

## Main Calculation

#### Saves Uploaded File and resets Widget

In [ ]:
# Triggered when: A file is uploaded

def SaveUpload(Widget_Verify_Upload):

    with open(UploadFileName + FileFormat, "w+b") as i:
        i.write(Widget_Upload.data[0])
    
    # Normalises Audio to -MaxGain - 3dB, that the maximum Output Amplification -3db is reached
    Normalise_Audio(UploadFileName, FileFormat, Normalise_Value)
    
    Widget_Upload.value.clear()
    Widget_Upload._counter = 0

#### Configuration Calculation

In [ ]:
# Triggered when: BandCount or FilterType is changed
EQDisplay = Build_EQ (20, Freqspecturmf0)

def ProcessConfiguration(SetFilterType, Widget_ChooseBandCount):
    global ChooseBandCount
    ChooseBandCount = Widget_ChooseBandCount
    # Calculate Base Frequency
    f_base = returnbasefreq(SetFilterType, ChooseBandCount)
    # Calculate f0 Frequencies
    global Freqspecturmf0
    Freqspecturmf0 = calculate_frequencies(SetFilterType, ChooseBandCount)
    # Calculates f0l
    global Freqspecturmfl
    Freqspecturmfl = calculate_frequencieslow(Freqspecturmf0, ChooseBandCount)
    # Calculates f0h
    global Freqspecturmfh
    Freqspecturmfh = calculate_frequencieshigh(Freqspecturmf0, ChooseBandCount)
    
    global ChoosenBandCount
    ChoosenBandCount = ChooseBandCount
    
    # Overwrites EQ Data
    global EQDisplay
    EQDisplay = Build_EQ (ChooseBandCount, Freqspecturmf0)
    ShowEQDisplay = widgets.Box(children = EQDisplay)
    display(ShowEQDisplay)
    
    # SaveValue = [0]*ChooseBandCount

    # for i in range(ChooseBandCount):
    #    SaveValue[i] = EQDisplay[i].value 

    # print(SaveValue)
    # display("Test")

#### EQ Calculation

In [ ]:
# Value that describes all steps of Progress

# Triggered 
def EQCalculation(Widget_Verify_EQ): 
    Output.clear_output()
    progress.value = 0
    Generate_Copies (CloneFileName, FileFormat, ChoosenBandCount)
    
    # Get EQ Values
    SaveValue = [0]*ChoosenBandCount
    for i in range(ChoosenBandCount):
        SaveValue[i] = EQDisplay[i].value
        
    # display(SaveValue)
    
    # Value of EQ Display is the Gain Value
    for n in range(0, ChoosenBandCount):
        Bandpass_Filter (CloneFileName, FileFormat, n, SaveValue[n], Freqspecturmfl[n], Freqspecturmfh[n])
        progress.value = float(n+1)/(ChoosenBandCount)
    
    Build_Audio (CloneFileName, FileFormat, FinishedFileName, ChoosenBandCount)
    
    waveUpload = read_wave(UploadFileName + ".wav")
    spectrum = waveUpload.make_spectrum()
    spectrum.plot(color='0.7')
    waveOutput = read_wave(FinishedFileName + ".wav")
    spectrum = waveOutput.make_spectrum()
    spectrum.plot(color='#045a8d')
    
    waveOutput.make_audio()
    
    with Output:
        plt.show()
        display(waveOutput.make_audio())

## User Interface

In [ ]:
if __name__ == "__main__":
    Output.clear_output()
    progress.value = 0
    Widget_Verify_Upload.on_click(SaveUpload)
    Configuration = widgets.interactive_output(ProcessConfiguration, {'SetFilterType' : Widget_SetFilterType, 'Widget_ChooseBandCount' : Widget_ChooseBandCount})
    Widget_Verify_EQ.on_click(EQCalculation)
    
    display(UserInterface, Configuration, Text5, Widget_Verify_EQ, progress, Output)

## Programmer

#### Programmed by Thomas Pail
    GitHub: https://github.com/Tomaru-Pai

## Formulas Used

## Sources

#### Example Audio File Credits
    Song: Dimension 
    Creator: Creo 
    Youtube: https://www.youtube.com/channel/UCsCWA3Y3JppL6feQiMRgm6Q 
    Website: https://creo-music.com/track/dimension
    Licensed under: https://creativecommons.org/licenses/by/4.0/

#### Programming
    https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
    https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
    https://en.wikibooks.org/wiki/LaTeX/Mathematics
    https://stackoverflow.com/

#### Formulars and Documents for Audio
    https://www.teachmeaudio.com/mixing/techniques/audio-spectrum#upper-midrange
    https://sound.stackexchange.com/questions/14101/what-is-the-frequency-step-formula-for-10-and-31-band-eqs
    http://www.sengpielaudio.com/calculator-octave.html
    https://github.com/AllenDowney/ThinkDSP

#### Other Programms used
    Music editing: Audacity
    Music Download: https://github.com/yt-dlp/yt-dlp

#### Special Thanks
    https://www.senarclens.eu/~gerald/